# Example of usage of an adaptative optimizer

## Variational Quantum State Eigensolver

In this notebook, we will optimize a VQSE problem. Here, we are given a set of states $\{\rho_i\}_{i=1}^N$ and we optimize the cost $L(\vec{\theta}) = \sum_{i} p_{i} E_{i}(\vec{\theta})$ where $\vec{\theta}$ are parameters of a variational circuit, and $E_i = \langle \rho_i |H| \rho_i \rangle, H = \mathbb{1} - r_j Z_j, r_j > 0$ .


In our example, $\{\rho_i\}_{i=1}^N$ will be taken from the collection of datasets available in Pennylane. We take several states obtained by running VQE for different bond lengths of the H2 molecule in the STO-3G basis. We obtain a dataset of 42 circuits as follows:

In [1]:
import pennylane as qml
from pennylane import numpy as np
from refoqus import Refoqus

bondlengths = ['0.5', '0.54', '0.58', '0.62', '0.66', '0.7', '0.74', '0.742', '0.78', '0.82', '0.86', '0.9', '0.94', '0.98', '1.02', '1.06', '1.1', '1.14', '1.18', '1.22', '1.26', '1.3', '1.34', '1.38', '1.42', '1.46', '1.5', '1.54', '1.58', '1.62', '1.66', '1.7', '1.74', '1.78', '1.82', '1.86', '1.9', '1.94', '1.98', '2.02', '2.06', '2.1']
moldataset = qml.data.load("qchem", molname="H2", basis="STO-3G", bondlength=bondlengths)
nbdatapoints = len(moldataset)

Now we set the coefficients r_j, the hamiltonian terms and even define the hamiltonian of interest $- r_j Z_j$ (note $1$ is a constant to be added later).

In [2]:
nbqbits = len(moldataset[0].hamiltonian.wires)
coefficients_cost = -np.array(
    [1.0 + (i - 1) * 0.2 for i in range(1, nbqbits + 1)]
)
coefficients_cost /= np.sum(coefficients_cost)

vqse_hamiltonian_term = [qml.PauliZ(i) for i in range(nbqbits)]

hamiltonian_of_interest = qml.Hamiltonian(coefficients_cost, vqse_hamiltonian_term)

Next, we define functions to evaluate the true cost during optimization.

In [3]:
analytic_dev = qml.device("default.qubit", wires=nbqbits, shots=None)

@qml.qnode(analytic_dev)
def cost_analytic_one_circuit(weights, index_datapoint):
    
    for op in moldataset[index_datapoint].vqe_gates:
        qml.apply(op)
        
    StronglyEntanglingLayers(weights, wires=analytic_dev.wires)
    return qml.expval(hamiltonian_of_interest)

def cost_analytic_alldataset(weights):
    
    cost = 0.0
    for m in range(nbdatapoints):
        cost += cost_analytic_one_circuit(weights, m)
    cost = 1.0 + cost / nbdatapoints
    return cost

Now, the ansatz is defined as with StronglyEntanglingLayers. We also sample initial values and the corresponding cost.

In [4]:
from pennylane.templates.layers import StronglyEntanglingLayers

# hyperparameter of ansatz
num_layers = 3


param_shape = StronglyEntanglingLayers.shape(n_layers=num_layers, n_wires=nbqbits)
np.random.seed(0)
init_params = np.random.uniform(low=0.0, high=2*np.pi, size=param_shape, requires_grad=True)
cost_analytic_alldataset(init_params)

tensor(0.8411577, requires_grad=True)

Our adaptative optimizer will be Refoqus where we provide the necessary arguments as follows and we perform niter iterations.

In [5]:
opt = Refoqus(nbqbits, [m.vqe_gates for m in moldataset], vqse_hamiltonian_term, coefficients_cost, param_shape, min_shots=2)
params = init_params
niter = 20

cost_refoqus = [cost_analytic_alldataset(params)]
shots_refoqus = [0]

for i in range(niter):
    params = opt.step(params)
    cost_refoqus.append(cost_analytic_alldataset(params))
    shots_refoqus.append(opt.shots_used)
    print(f"Step {i+1}: cost = {cost_refoqus[-1]}, shots_used = {shots_refoqus[-1]}")

Step 1: cost = 0.9873139229394886, shots_used = 144
Step 2: cost = 1.0463301131567824, shots_used = 728
Step 3: cost = 0.9658541188174995, shots_used = 1384
Step 4: cost = 0.9458409425041674, shots_used = 2204
Step 5: cost = 0.741037027856992, shots_used = 2926
Step 6: cost = 0.6968740272326923, shots_used = 4106
Step 7: cost = 0.6225473428363348, shots_used = 6614
Step 8: cost = 0.5066941409872253, shots_used = 9840
Step 9: cost = 0.41150925020991846, shots_used = 14730
Step 10: cost = 0.3258301607842782, shots_used = 21224
Step 11: cost = 0.2569035497396862, shots_used = 30360
Step 12: cost = 0.2194796776824891, shots_used = 40532
Step 13: cost = 0.1675813922495496, shots_used = 57850
Step 14: cost = 0.14341482799894611, shots_used = 70414
Step 15: cost = 0.10663086753664364, shots_used = 90442
Step 16: cost = 0.08005878665815014, shots_used = 108644
Step 17: cost = 0.06299584623523613, shots_used = 129540
Step 18: cost = 0.03919438495816574, shots_used = 158452
Step 19: cost = 0.028